# Encriptacion y Desencriptacion usando ECC

### Herramientas usadas: python 3, ECC como generador de llave publica y privada y AES como metodo de encriptacion y desencriptacion

#### Explicacion:
El codigo lo que hace es generar valores enteros entre 23 y 37, simulando ser datos de temperaturas obtenidas por el sensor, que luego se almacenan en un bloc de notas de nombre "temperaturas.txt" que tambien es creado por el codigo.
Luego se definen las funciones de encriptacion y desencriptacion con AES-GCM

In [5]:
import random
import time
import requests
import io
import git
from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii
import pickle 

In [6]:
with open("/Users/Chris-OSX/Cosas/temperaturas.txt", "w") as w:
    for n in range(0, 100):
        w.write(str(random.randint(23, 37))+ " °C\n")
    

In [7]:
def encrypt_AES_GCM(msg, secretKey):
  aesCipher = AES.new(secretKey, AES.MODE_GCM)
  ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
  return (ciphertext, aesCipher.nonce, authTag)


def ecc_point_to_256_bit_key(point):
  sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
  sha.update(int.to_bytes(point.y, 32, 'big'))
  return sha.digest()

curve = registry.get_curve('brainpoolP256r1')

def encrypt_ECC(msg, pubKey):
  ciphertextPrivKey = secrets.randbelow(curve.field.n) # a
  sharedECCKey = ciphertextPrivKey * pubKey #a·B = T
  secretKey = ecc_point_to_256_bit_key(sharedECCKey)
  ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey)
  ciphertextPubKey = ciphertextPrivKey * curve.g #A
  return (ciphertext, nonce, authTag, ciphertextPubKey)


In [11]:
f = open("/Users/Chris-OSX/Cosas/temperaturas.txt", "rb")
repo = git.Repo("/Users/Chris-OSX/Cosas/TEL252_Central_de_datos")

while(True):
    msg = f.readline()
    print("original msg:", msg.decode('UTF-8'))

    time.sleep(3)
    repo.remotes.origin.pull()

    with open('pub_keyB.pickle', 'rb') as f:
      pubKeyB = pickle.load(f) #B

    encryptedMsg = encrypt_ECC(msg, pubKeyB)
    with open('encrypt_data.pickle', 'wb') as f:
      pickle.dump(encryptedMsg, f)

    encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'nonce': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2]),
    'ciphertextPubKey': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
    }
    print("encrypted msg:", encryptedMsgObj)
    
    repo.git.add(repo.working_dir)
    commit_output = repo.git.commit(m="Data Update")
    push_output = repo.git.push('--set-upstream', repo.remote().name, "main")

    if not msg:
        break
f.close()


original msg: 35 °C

encrypted msg: {'ciphertext': b'e98a89874c3d2f', 'nonce': b'75809a36dd4abdaae457bc6833a41bc5', 'authTag': b'5f06f5619a44d37b7ea1ec31bbca717f', 'ciphertextPubKey': '0x9dcfd57c8683d512c79aa7c09f5867b9790d561d1303d7046676842a4c8fed4f1'}


UnsupportedOperation: read

In [ ]:
repo.remotes.origin.pull()

In [ ]:
#!git commit -m "Test 1"

In [ ]:
#!git config --global user.email "christopher.silva@sansano.usm.cl"

In [ ]:
#!git config --global user.name "Chrismsz"

In [ ]:
!cd TEL252_Central_de_datos

In [ ]:
repo.git.add(repo.working_dir)
commit_output = repo.git.commit(m="Data Update")
push_output = repo.git.push('--set-upstream', repo.remote().name, "main")

In [ ]:
cd ..

In [ ]:
!git remote set-url origin https://{Chrismsz}:{Esta pass es para git}@github.com/{Chrismsz}/https://github.com/Chrismsz/TEL252_Central_de_datos.git